## Парсинг данных

* Соберем все стихотворения Корнея Чуковского на сайте 'https://rustih.ru/kornej-chukovskij/'.

### **Работа с библиотекой beautifulsoup4**

In [1]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


Импорт библиотек для осуществления запросов и парсинга, а также для работы с таблицами

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'https://rustih.ru/kornej-chukovskij/'

response = requests.get(url)

if response.status_code == 200:
    print('Запрос прошел успешно!')
else:
    print(f'Ошибка! Response code: {response.status_code}')
    print(f'URL: {url}')
    print('--------------')

Запрос прошел успешно!


Иногда владельцы сайтов хотят помешать другим парсить сайт. Для этого вводятся дополнительные проверки запроса на сервер.

Самая частая проверка состоит в извлечении информации об используемом браузере с заголовка запроса.
Чтобы сказать сайту, что мы "браузер", добавим информацию об user_agent в наш скрипт.

Такие заголовки можно найти на просторах сети, или посмотреть в конфигурациях браузера.

In [4]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

response = requests.get(url, headers=headers, allow_redirects=True)

page_text = ''
if response.status_code == 200:
    print('Запрос прошел успешно!')
    page_text = response.text
else:
    print(f'Ошибка! Response code: {response.status_code}')
    print(f'URL: {url}')
    print('--------------')

Запрос прошел успешно!


In [5]:
print(page_text[:1000])


<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta http-equiv="x-dns-prefetch-control" content="on">
<link href="//googleads.g.doubleclick.net" rel="dns-prefetch" />
<link href="//pagead2.googlesyndication.com" rel="dns-prefetch" />
<link href="//ajax.googleapis.com" rel="dns-prefetch">
<link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
<link href="//googleads.g.doubleclick.net" rel="preconnect" />
<link href="//pagead2.googlesyndication.com" rel="preconnect" />
<link href="//ajax.googleapis.com" rel="preconnect">
<link href="//tpc.googlesyndication.com" rel="preconnect" />
<script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<link rel="shortcut icon" href="https://rustih.ru/favicon.ico" type="image/x-icon" />
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Корней Чуковский стихи: читать стихотворения, произведения Корнея Ивановича Чуковского - Стишки, поэзия поэта

In [6]:
page_bs = BeautifulSoup(page_text)

In [7]:
coverpage = page_bs.find('div', {'id': 'primary'})

In [8]:
shelf_containers = coverpage.find_all('div', {'class': ['post-card-one']})

print(f'Количество найденных блоков: {len(shelf_containers)}')

Количество найденных блоков: 42


**Извлечем название стихотворения и название ссылки на него, чтобы затем по этой ссылке получить текст самого стихотворения**

In [9]:
title_list = []
url_list = []
i = 0
for shelf in shelf_containers:
    # Извлечение названия заголовка
    title_container = shelf.find('div', {'class': 'entry-title'})
    title_text = title_container.find('a').text
    title_list.append(title_text)
    
    # Извлечение ссылки
    url_text = title_container.find('a')['href']
    url_list.append(url_text)
    
print(f'Заголовок: {title_text}')
print(f'Ссылка: {url_text}')
print('------------\n')

Заголовок: Корней Чуковский — Перед праздником зима
Ссылка: https://rustih.ru/kornej-chukovskij-pered-prazdnikom-zima/
------------



Запишем полученные результаты в датафрейм, чтобы было удобно работать с данными

In [10]:
url_df = pd.DataFrame({'title': title_list, 'url': url_list})
url_df.head(5)

,title,url
0,Корней Чуковский — Айболит,https://rustih.ru/kornej-chukovskij-ajbolit/
1,Корней Чуковский — Мойдодыр,https://rustih.ru/kornej-chukovskij-mojdodyr/
2,Корней Чуковский — Краденое солнце,https://rustih.ru/kornej-chukovskij-kradenoe-s...
3,Корней Чуковский — Чудо-дерево,https://rustih.ru/kornej-chukovskij-chudo-derevo/
4,Корней Чуковский — Бармалей,https://rustih.ru/kornej-chukovskij-barmalej/


Результаты можно сохранить в файл, чтобы не потерять :) 

In [11]:
url_df.to_csv('poems_url.csv', index=False)
#url_df.to_excel('themes_url.xls', index=False)

**По ссылкам парсим текст стихотворений**

In [12]:
titles = []
poems = []
for title, url in url_df.values:
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}

    response = requests.get(url, headers=headers, allow_redirects=True)

    page_text = ''
    if response.status_code == 200:
        #print('Запрос прошел успешно!')
        page_text = response.text
    else:
        print(f'Ошибка! Response code: {response.status_code}')
        print(f'URL: {url}')
        print('--------------')
    
    page_parser = BeautifulSoup(page_text)
    attraction_list = page_parser.find('main', {'id': ['main']})
        
    header = attraction_list.find('header', {'class': ['entry-header']})
    t = header.find('h1').text
    titles.append(t)

    text_poems_list = attraction_list.find('div', {'class': ['entry-content poem-text']})
    text_poems_list = text_poems_list.find_all('p')
    poems.append(text_poems_list)

df = pd.DataFrame({'title': titles, 'text': poems})
df.to_csv('title_text.csv', index=False)   

Получили таблицу с названием стихотворения и его текстом

In [13]:
df.head()

,title,text
0,Корней Чуковский — Айболит: Стих,"[[1], [Добрый доктор Айболит!, [], \nОн под де..."
1,Корней Чуковский — Мойдодыр: Стих,"[[Одеяло, [], \nУбежало,, [], \nУлетела просты..."
2,Корней Чуковский — Краденое солнце: Стих,"[[Солнце по небу гуляло, [], \nИ за тучу забеж..."
3,Корней Чуковский — Чудо-дерево: Стих,"[[Как у нашего Мирона, [], \nНа носу сидит вор..."
4,Корней Чуковский — Бармалей: Стих,"[[I], [Маленькие дети!, [], \nНи за что на све..."


In [14]:
df.shape

(42, 2)

Всего 42 стихотворения

### **Почистим текст для удобной работы с ним вдальнейшем**

* Удаляем пробелы вначале и вконце строк

In [15]:
df['title'] = df['title'].str.strip()
df.head()

,title,text
0,Корней Чуковский — Айболит: Стих,"[[1], [Добрый доктор Айболит!, [], \nОн под де..."
1,Корней Чуковский — Мойдодыр: Стих,"[[Одеяло, [], \nУбежало,, [], \nУлетела просты..."
2,Корней Чуковский — Краденое солнце: Стих,"[[Солнце по небу гуляло, [], \nИ за тучу забеж..."
3,Корней Чуковский — Чудо-дерево: Стих,"[[Как у нашего Мирона, [], \nНа носу сидит вор..."
4,Корней Чуковский — Бармалей: Стих,"[[I], [Маленькие дети!, [], \nНи за что на све..."


* Чистим текст от ненужныхсимволов, разбиваем на отдельные элементы списка

In [16]:
def clear_text(text):
    new_text = ','.join(text.split('—'))
    new_text = ','.join(new_text.split(': '))
    return new_text.split(',')

df['title'] = df['title'].apply(clear_text)
display(df.head())

,title,text
0,"[Корней Чуковский , Айболит, Стих]","[[1], [Добрый доктор Айболит!, [], \nОн под де..."
1,"[Корней Чуковский , Мойдодыр, Стих]","[[Одеяло, [], \nУбежало,, [], \nУлетела просты..."
2,"[Корней Чуковский , Краденое солнце, Стих]","[[Солнце по небу гуляло, [], \nИ за тучу забеж..."
3,"[Корней Чуковский , Чудо-дерево, Стих]","[[Как у нашего Мирона, [], \nНа носу сидит вор..."
4,"[Корней Чуковский , Бармалей, Стих]","[[I], [Маленькие дети!, [], \nНи за что на све..."


* Выделяем отдельные столбцы "автор" и "название стихотворения"

In [17]:
def split_list(list, i):
    text = list[i]
    return text.strip()
df['author'] = df['title'].apply(split_list, i=0)
df['poem_title'] = df['title'].apply(split_list, i=1)
display(df.head())

,title,text,author,poem_title
0,"[Корней Чуковский , Айболит, Стих]","[[1], [Добрый доктор Айболит!, [], \nОн под де...",Корней Чуковский,Айболит
1,"[Корней Чуковский , Мойдодыр, Стих]","[[Одеяло, [], \nУбежало,, [], \nУлетела просты...",Корней Чуковский,Мойдодыр
2,"[Корней Чуковский , Краденое солнце, Стих]","[[Солнце по небу гуляло, [], \nИ за тучу забеж...",Корней Чуковский,Краденое солнце
3,"[Корней Чуковский , Чудо-дерево, Стих]","[[Как у нашего Мирона, [], \nНа носу сидит вор...",Корней Чуковский,Чудо-дерево
4,"[Корней Чуковский , Бармалей, Стих]","[[I], [Маленькие дети!, [], \nНи за что на све...",Корней Чуковский,Бармалей


In [18]:
df.columns

Index(['title', 'text', 'author', 'poem_title'], dtype='object')

* Создаем таблицу с нужными столбцами и сохраняем ее

In [19]:
df = df.loc[:, ['text', 'poem_title']]

In [20]:
df.head()

,text,poem_title
0,"[[1], [Добрый доктор Айболит!, [], \nОн под де...",Айболит
1,"[[Одеяло, [], \nУбежало,, [], \nУлетела просты...",Мойдодыр
2,"[[Солнце по небу гуляло, [], \nИ за тучу забеж...",Краденое солнце
3,"[[Как у нашего Мирона, [], \nНа носу сидит вор...",Чудо-дерево
4,"[[I], [Маленькие дети!, [], \nНи за что на све...",Бармалей


In [21]:
df.to_csv('poems_title.csv', index=False)

### Парсинг данных осуществлен успешно. Найдены необходимые данные.